# Inspecting Ecore file for SysML v2 Metamodel

This notebook steps through a couple of different methods for loading up the SysML v2 Ecore metamodel. One with a library called pyEcore and another just using the raw lxml library.

In [1]:
import lxml

In [2]:
from pyecore.resources import ResourceSet, URI
from pyecore.ecore import EClass, EAttribute, EString, EObject, EProxy, EEnum

In [3]:
# From the quick start for PyEcore to load a file into the metaclass model

rset = ResourceSet()
sysml_resource = rset.get_resource(URI('metamodel/SysML.ecore'))
mm_root = sysml_resource.contents[0]
rset.metamodel_registry[mm_root.nsURI] = mm_root

types_resource = rset.get_resource(URI('metamodel/types.ecore'))
mm_root2 = types_resource.contents[0]
rset.metamodel_registry[mm_root.nsURI] = mm_root2

In [4]:
mm_root

Determine what the fields for an EPackage in this framework are:

In [5]:
dir(mm_root)

['eAnnotations',
 'eClassifiers',
 'eSubpackages',
 'eSuperPackage',
 'eURIFragment',
 'getEAnnotation',
 'getEClassifier',
 'name',
 'nsPrefix',
 'nsURI']

In [6]:
all_classes = mm_root.eClassifiers

Determine what the fields for an EClass in this framework are:

In [7]:
dir(all_classes[0])

['abstract',
 'allInstances',
 'defaultValue',
 'eAllAttributes',
 'eAllGenericSuperTypes',
 'eAllGenericSuperTypesClassifiers',
 'eAllOperations',
 'eAllReferences',
 'eAllStructuralFeatures',
 'eAllSuperTypes',
 'eAnnotations',
 'eAttributes',
 'eGenericSuperTypes',
 'eOperations',
 'ePackage',
 'eReferences',
 'eStructuralFeatures',
 'eSuperTypes',
 'eTypeParameters',
 'eURIFragment',
 'findEOperation',
 'findEStructuralFeature',
 'getEAnnotation',
 'instanceClass',
 'instanceClassName',
 'instanceTypeName',
 'interface',
 'name',
 'notifyChanged']

In [8]:
def get_eclass(eclass_name):
    return [the_class for the_class in all_classes if the_class.name == eclass_name][0]

In [9]:
owning_membership = [the_class for the_class in all_classes if the_class.name == "OwningMembership"]

In [10]:
owning_membership[0]

<EClass name="OwningMembership">

In [11]:
owning_membership[0].eAttributes

[<EAttribute ownedMemberElementId: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>,
 <EAttribute ownedMemberShortName: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>,
 <EAttribute ownedMemberName: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>]

In [12]:
owning_membership[0].eReferences

[<EReference ownedMemberElement: <EClass name="Element">>]

In [13]:
owning_membership[0].eAttributes[0]

<EAttribute ownedMemberElementId: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>

In [14]:
dir(owning_membership[0].eAttributes[0])

['changeable',
 'defaultValueLiteral',
 'derived',
 'eAnnotations',
 'eContainingClass',
 'eGenericType',
 'eType',
 'eURIFragment',
 'getEAnnotation',
 'get_default_value',
 'iD',
 'lower',
 'lowerBound',
 'name',
 'notifyChanged',
 'notifyChanged',
 'ordered',
 'required',
 'transient',
 'unique',
 'unsettable',
 'upper',
 'upperBound',
 'volatile']

In [15]:
owning_membership[0].eAttributes[0].derived

True

In [16]:
attr_type = None
if isinstance(owning_membership[0].eAttributes[0].eType, EProxy):
    attr_type = owning_membership[0].eAttributes[0].eType.force_resolve()
else:
    attr_type = owning_membership[0].eAttributes[0].eType

In [17]:
attr_type.name

'String'

In [18]:
[super_type.name for super_type in all_classes[0].eAllSuperTypes()]

['StateUsage',
 'PerformActionUsage',
 'ActionUsage',
 'OccurrenceUsage',
 'Step',
 'Usage',
 'Feature',
 'Type',
 'Namespace',
 'Element',
 'EventOccurrenceUsage']

In [19]:
owning_membership[0].eAllAttributes()

{<EAttribute aliasIds: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>,
 <EAttribute declaredName: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>,
 <EAttribute declaredShortName: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>,
 <EAttribute elementId: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>,
 <EAttribute isImplied: <pyecore.ecore.EProxy object at 0x000002753B4B4AF0>>,
 <EAttribute isImpliedIncluded: <pyecore.ecore.EProxy object at 0x000002753B4B4AF0>>,
 <EAttribute isLibraryElement: <pyecore.ecore.EProxy object at 0x000002753B4B4AF0>>,
 <EAttribute memberElementId: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>,
 <EAttribute memberName: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>,
 <EAttribute memberShortName: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>,
 <EAttribute name: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>,
 <EAttribute ownedMemberElementId: <pyecore.ecore.EProxy object at 0x000002753B4B4EB0>>,
 <EAttribute ownedMe

In [20]:
owning_membership[0].eAllReferences()

{<EReference documentation: <EClass name="Documentation">>,
 <EReference memberElement: <EClass name="Element">>,
 <EReference membershipOwningNamespace: <EClass name="Namespace">>,
 <EReference ownedAnnotation: <EClass name="Annotation">>,
 <EReference ownedElement: <EClass name="Element">>,
 <EReference ownedMemberElement: <EClass name="Element">>,
 <EReference ownedRelatedElement: <EClass name="Element">>,
 <EReference ownedRelationship: <EClass name="Relationship">>,
 <EReference owner: <EClass name="Element">>,
 <EReference owningMembership: <EClass name="OwningMembership">>,
 <EReference owningNamespace: <EClass name="Namespace">>,
 <EReference owningRelatedElement: <EClass name="Element">>,
 <EReference owningRelationship: <EClass name="Relationship">>,
 <EReference relatedElement: <EClass name="Element">>,
 <EReference source: <EClass name="Element">>,
 <EReference target: <EClass name="Element">>,
 <EReference textualRepresentation: <EClass name="TextualRepresentation">>}

In [21]:
def element_has_feature_by_name(element, name):
    all_refs = element.eAllReferences()
    all_atts = element.eAllAttributes()
    
    for ref in all_refs:
        if ref.name == name:
            return True
        
    for att in all_atts:
        if att.name == name:
            return True
        
    return False

In [22]:
element_has_feature_by_name(owning_membership[0], "relatedElement")

True

In [23]:
def layout_features(element):
    
    feature_line = []
    
    all_refs = element.eAllReferences()
    all_atts = element.eAllAttributes()
    
    local_refs = element.eReferences
    local_atts = element.eAttributes
    
    for ref in all_refs:
        new_line = []
        new_line.append(ref.name)
        if ref in local_refs:
            new_line.append('local')
        else:
            new_line.append('inherited')
        if ref.derived:
            new_line.append('derived')
        else:
            new_line.append('primary')
        if isinstance(ref.eType, EProxy):
            new_line.append(ref.eType.force_resolve().name)
        else:
            new_line.append(ref.eType.name)
        
        new_line.append('EReference')
        feature_line.append(new_line)
        
    for att in all_atts:
        new_line = []
        new_line.append(att.name)
        if att in local_atts:
            new_line.append('local')
        else:
            new_line.append('inherited')
        if att.derived:
            new_line.append('derived')
        else:
            new_line.append('primary')
        if isinstance(att.eType, EProxy):
            new_line.append(att.eType.force_resolve().name)
        else:
            new_line.append(att.eType.name)
        
        new_line.append('EAttribute')
        feature_line.append(new_line)
    
    return feature_line

In [24]:
layout_features(owning_membership[0])

[['ownedMemberElement', 'local', 'derived', 'Element', 'EReference'],
 ['relatedElement', 'inherited', 'derived', 'Element', 'EReference'],
 ['target', 'inherited', 'primary', 'Element', 'EReference'],
 ['memberElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['owningMembership',
  'inherited',
  'derived',
  'OwningMembership',
  'EReference'],
 ['owner', 'inherited', 'derived', 'Element', 'EReference'],
 ['ownedRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['textualRepresentation',
  'inherited',
  'derived',
  'TextualRepresentation',
  'EReference'],
 ['owningRelatedElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['membershipOwningNamespace',
  'inherited',
  'derived',
  'Namespace',
  'EReference'],
 ['ownedElement', 'inherited', 'derived', 'Element', 'EReference'],
 ['ownedAnnotation', 'inherited', 'derived', 'Annotation', 'EReference'],
 ['source', 'inherited', 'primary', 'Element', 'EReference'],
 ['owningNamespace', 'inherited

In [31]:
def get_primary_attributes(eclass_name):
    eclass = get_eclass(eclass_name)
    
    all_features = layout_features(eclass)
    
    primary_features = [feature for feature in all_features if feature[2] == 'primary']
    
    return primary_features

In [25]:
layout_features(get_eclass("OwningMembership"))

[['ownedMemberElement', 'local', 'derived', 'Element', 'EReference'],
 ['relatedElement', 'inherited', 'derived', 'Element', 'EReference'],
 ['target', 'inherited', 'primary', 'Element', 'EReference'],
 ['memberElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['owningMembership',
  'inherited',
  'derived',
  'OwningMembership',
  'EReference'],
 ['owner', 'inherited', 'derived', 'Element', 'EReference'],
 ['ownedRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['textualRepresentation',
  'inherited',
  'derived',
  'TextualRepresentation',
  'EReference'],
 ['owningRelatedElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['membershipOwningNamespace',
  'inherited',
  'derived',
  'Namespace',
  'EReference'],
 ['ownedElement', 'inherited', 'derived', 'Element', 'EReference'],
 ['ownedAnnotation', 'inherited', 'derived', 'Annotation', 'EReference'],
 ['source', 'inherited', 'primary', 'Element', 'EReference'],
 ['owningNamespace', 'inherited

In [37]:
layout_features(get_eclass("PartUsage"))

[['nestedAttribute', 'inherited', 'derived', 'AttributeUsage', 'EReference'],
 ['nestedConstraint', 'inherited', 'derived', 'ConstraintUsage', 'EReference'],
 ['ownedFeatureInverting',
  'inherited',
  'derived',
  'FeatureInverting',
  'EReference'],
 ['owner', 'inherited', 'derived', 'Element', 'EReference'],
 ['nestedOccurrence', 'inherited', 'derived', 'OccurrenceUsage', 'EReference'],
 ['ownedRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['endOwningType', 'inherited', 'derived', 'Type', 'EReference'],
 ['inheritedMembership', 'inherited', 'derived', 'Membership', 'EReference'],
 ['inheritedFeature', 'inherited', 'derived', 'Feature', 'EReference'],
 ['partDefinition', 'local', 'derived', 'PartDefinition', 'EReference'],
 ['input', 'inherited', 'derived', 'Feature', 'EReference'],
 ['nestedReference', 'inherited', 'derived', 'ReferenceUsage', 'EReference'],
 ['ownedDifferencing', 'inherited', 'derived', 'Differencing', 'EReference'],
 ['nestedAction', 'inhe

In [27]:
def find_all_sub_types(element):
    # go through all of the meta-elements in the Ecore and find the ones that have the 
    # element given as an argument as one of their super types
    
    sub_types = []
    
    all_classes = mm_root.eClassifiers
    for clz in all_classes:
        if not isinstance(clz, EEnum):
            if element in clz.eAllSuperTypes():
                sub_types.append(clz)
            
    return sub_types

In [28]:
find_all_sub_types(get_eclass("OwningMembership"))

[<EClass name="FeatureMembership">,
 <EClass name="VariantMembership">,
 <EClass name="TransitionFeatureMembership">,
 <EClass name="StateSubactionMembership">,
 <EClass name="ViewRenderingMembership">,
 <EClass name="ObjectiveMembership">,
 <EClass name="RequirementVerificationMembership">,
 <EClass name="RequirementConstraintMembership">,
 <EClass name="SubjectMembership">,
 <EClass name="ParameterMembership">,
 <EClass name="StakeholderMembership">,
 <EClass name="FramedConcernMembership">,
 <EClass name="ActorMembership">,
 <EClass name="FeatureValue">,
 <EClass name="ReturnParameterMembership">,
 <EClass name="ResultExpressionMembership">,
 <EClass name="ElementFilterMembership">,
 <EClass name="EndFeatureMembership">]

In [32]:
get_primary_attributes("PartDefinition")

[['ownedRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['owningRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['isImpliedIncluded', 'inherited', 'primary', 'Boolean', 'EAttribute'],
 ['elementId', 'inherited', 'primary', 'String', 'EAttribute'],
 ['isSufficient', 'inherited', 'primary', 'Boolean', 'EAttribute'],
 ['isVariation', 'inherited', 'primary', 'Boolean', 'EAttribute'],
 ['isAbstract', 'inherited', 'primary', 'Boolean', 'EAttribute'],
 ['declaredShortName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['isIndividual', 'inherited', 'primary', 'Boolean', 'EAttribute'],
 ['declaredName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['aliasIds', 'inherited', 'primary', 'String', 'EAttribute']]

In [33]:
get_primary_attributes("OwningMembership")

[['target', 'inherited', 'primary', 'Element', 'EReference'],
 ['memberElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['ownedRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['owningRelatedElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['source', 'inherited', 'primary', 'Element', 'EReference'],
 ['ownedRelatedElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['owningRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['isImpliedIncluded', 'inherited', 'primary', 'Boolean', 'EAttribute'],
 ['memberName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['elementId', 'inherited', 'primary', 'String', 'EAttribute'],
 ['memberShortName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['declaredShortName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['declaredName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['aliasIds', 'inherited', 'primary', 'String', 'EAttribute'],
 ['visibility', 'inherite

In [34]:
get_primary_attributes("Membership")

[['target', 'inherited', 'primary', 'Element', 'EReference'],
 ['memberElement', 'local', 'primary', 'Element', 'EReference'],
 ['ownedRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['owningRelatedElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['source', 'inherited', 'primary', 'Element', 'EReference'],
 ['ownedRelatedElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['owningRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['isImpliedIncluded', 'inherited', 'primary', 'Boolean', 'EAttribute'],
 ['memberName', 'local', 'primary', 'String', 'EAttribute'],
 ['elementId', 'inherited', 'primary', 'String', 'EAttribute'],
 ['memberShortName', 'local', 'primary', 'String', 'EAttribute'],
 ['declaredShortName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['declaredName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['aliasIds', 'inherited', 'primary', 'String', 'EAttribute'],
 ['visibility', 'local', 'primary', '

In [35]:
get_primary_attributes("Specialization")

[['source', 'inherited', 'primary', 'Element', 'EReference'],
 ['target', 'inherited', 'primary', 'Element', 'EReference'],
 ['ownedRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['owningRelatedElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['specific', 'local', 'primary', 'Type', 'EReference'],
 ['general', 'local', 'primary', 'Type', 'EReference'],
 ['ownedRelatedElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['owningRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['isImpliedIncluded', 'inherited', 'primary', 'Boolean', 'EAttribute'],
 ['elementId', 'inherited', 'primary', 'String', 'EAttribute'],
 ['declaredShortName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['declaredName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['aliasIds', 'inherited', 'primary', 'String', 'EAttribute'],
 ['isImplied', 'inherited', 'primary', 'Boolean', 'EAttribute']]

In [36]:
get_primary_attributes("FeatureTyping")

[['source', 'inherited', 'primary', 'Element', 'EReference'],
 ['typedFeature', 'local', 'primary', 'Feature', 'EReference'],
 ['ownedRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['specific', 'inherited', 'primary', 'Type', 'EReference'],
 ['ownedRelatedElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['target', 'inherited', 'primary', 'Element', 'EReference'],
 ['general', 'inherited', 'primary', 'Type', 'EReference'],
 ['owningRelationship', 'inherited', 'primary', 'Relationship', 'EReference'],
 ['owningRelatedElement', 'inherited', 'primary', 'Element', 'EReference'],
 ['type', 'local', 'primary', 'Type', 'EReference'],
 ['isImpliedIncluded', 'inherited', 'primary', 'Boolean', 'EAttribute'],
 ['elementId', 'inherited', 'primary', 'String', 'EAttribute'],
 ['declaredShortName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['declaredName', 'inherited', 'primary', 'String', 'EAttribute'],
 ['aliasIds', 'inherited', 'primary', 'String', 'EAttr